In [40]:

import numpy as np
import pandas as pd

In [41]:
df = pd.read_csv('gurgaon_properties_post_feature_selection.csv')

In [42]:

df.head()

,property_type,sector,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category,price
0,1.0,15.0,8.0,5.0,2.0,0.0,2476.0,0,0,0,1.0,1.0,1.80
1,1.0,55.0,6.0,5.0,4.0,0.0,6000.0,0,0,0,2.0,2.0,3.35
2,0.0,61.0,3.0,4.0,4.0,1.0,2895.0,0,0,0,2.0,0.0,4.80
3,0.0,45.0,2.0,3.0,2.0,3.0,656.0,0,0,0,1.0,2.0,0.45
4,1.0,84.0,5.0,5.0,4.0,3.0,3653.0,0,0,0,1.0,0.0,3.10


In [43]:
# one hot encode -> sector, balcony, agePossession, furnishing type, luxury category, floor category

In [44]:
X = df.drop(columns=['price'])
y = df['price']

In [45]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR


In [46]:
# Applying the log1p transformation to the target variable
y_transformed = np.log1p(y)

In [47]:
# Categorical columns to encode
columns_to_encode = ['sector', 'balcony', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']

# Preprocessing pipeline
preprocessor = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(handle_unknown='ignore'), columns_to_encode),
    ('num', StandardScaler(), [col for col in X.columns if col not in columns_to_encode])
    ])


In [48]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', SVR(kernel='rbf'))
])

In [49]:

# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

In [50]:
scores.mean()

np.float64(0.882648131281892)

In [51]:
scores.std()

np.float64(0.02221886296149019)

In [52]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)

In [53]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['sector', 'balcony',
                                                   'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category']),
                                                 ('num', StandardScaler(),
                                                  ['property_type', 'bedRoom',
                                                   'bathroom', 'built_up_area',
                                                   'servant room',
                                                   'store room'])])),
                ('regressor', SVR())])

In [54]:
y_pred = pipeline.predict(X_test)

In [57]:
y_pred = np.expm1(y_pred) # this step is written to get y which was previously converted into log y

In [56]:

from sklearn.metrics import mean_absolute_error
mean_absolute_error(np.expm1(y_test),y_pred)

0.5535842176285931